# LFVI for conditional density estimation

Notebook for testing LFVI as a tool for non-parametric posterior densitiy estimation in ABC

Notation:
- $x$ is the observed variable and corresponds to the summary statistics used in ABC
- $z$ is the latent variable and corresponds to the simulator parameter ( '' $\theta$ '' ) in ABC

We seek to learn $q(z|x) \approx p(z|x)$ for prior $p(z)$ and some imlicitly defined likelihood $p(x|z)$

In [ ]:
import edward as ed
import numpy as np
import tensorflow as tf

ed.set_seed(42)

# define experiment, generate toy data

In [ ]:
N, M = 1000, 100  # batch size during training
D, K = 1, 1       # dimensionality of observed and hidden variables

# prior 
def gen_z(N):
    return 3 * (np.random.uniform(size=(N,K))-0.5)

# simulator (induces implicit likelihood)
def gen_x(z):
    N = z.shape[0]
    return np.exp(z)/2. - 1/2. + np.random.normal(size=(N,D))/16.

# train and test data (test data only used for visualization)
z_train, z_test = gen_z(N), gen_z(1000)
x_train, x_test = gen_x(z_train), gen_x(z_test)

# translate experiment to Edward

In [ ]:
from edward.models import Normal, Uniform, PointMass

# define log-ratio estimator r(x,z) (has to be flexible enough or nothing works!)
def discriminative_network(xdict, zdict, betadict):
    """Outputs probability in logits."""
    net = tf.layers.dense(tf.concat([xdict[x], zdict[z]], 1), 64, activation=tf.nn.relu)
    net = tf.layers.dense(net, 64, activation=tf.nn.relu)
    net = tf.layers.dense(net, 64, activation=tf.nn.relu)
    net = tf.layers.dense(net, 1, activation=None)
    return net

# define simple generative model in Edward (has to match data-generation mechanism above)
z = Uniform(low=-1.5*tf.ones([M,K]), high=1.5*tf.ones([M,K])) 
x = Normal(loc= tf.exp(z)/2. - 0.5, scale=tf.ones([M,D])/16.)    # p(x|z)

x_ph = tf.placeholder(tf.float32, [M, D]) # container for x_train
z_ph = tf.placeholder(tf.float32, [M, K]) # container for z_train

# define simple flexible recognition model q(z|x)
def generative_network(eps):
    net = tf.layers.dense(eps, 5, activation=tf.nn.tanh)
    net = tf.layers.dense(net, 5, activation=tf.nn.tanh)
    net = tf.layers.dense(net, D, activation=None)
    return net
# 'single-component MDN with fixed covariance'
qz = Normal(loc=generative_network(x_ph), scale=tf.ones([M,D])/10.)  


# some Edward / tensorflow

In [ ]:
import six
from edward.util import check_latent_vars, copy, get_session

## assemble elements of tensorflow graph 

scope = tf.get_default_graph().unique_name("inference")
discriminator=discriminative_network
qbeta_sample = {}
x_qsample, x_psample = {x : x_ph},        {x: copy(x, dict_swap=qbeta_sample, scope=scope).value()}
qz_sample, pz_sample = {z: qz.value()},   {z: copy(z, dict_swap=qbeta_sample, scope=scope).value()}

x_ph_t = tf.placeholder(tf.float32, [1000, D]) # container for x_test
z_ph_t = tf.placeholder(tf.float32, [1000, K]) # container for z_train
px_test, pz_test = {x: x_ph_t}, {z: z_ph_t}

with tf.variable_scope("Disc"):
    r_qsample = discriminator(x_qsample, qz_sample, qbeta_sample) # see implicit_klqp
with tf.variable_scope("Disc", reuse=True):
    r_psample = discriminator(x_psample, pz_sample, qbeta_sample) # for original usage
with tf.variable_scope("Disc", reuse=True):
    r_test = discriminator(px_test, pz_test, qbeta_sample)          # for querying the log-ratio density net
    

## construct losses and optimizers
    
# loss and trainer for ratio-density network     
def log_loss(psample, qsample):
    """Point-wise log loss."""
    loss = tf.nn.sigmoid_cross_entropy_with_logits(
        labels=tf.ones_like(psample), logits=psample) + \
        tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(qsample), logits=qsample)
    return loss
ratio_loss = log_loss
loss_d = tf.reduce_mean(ratio_loss(r_psample, r_qsample)) 

var_list_d = tf.get_collection(
tf.GraphKeys.TRAINABLE_VARIABLES, scope="Disc")
print('var_list_d \n', var_list_d)

optimizer_d = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9) # optimizer for r(x,z)

grads_d = tf.gradients(loss_d, var_list_d)
grads_and_vars_d = list(zip(grads_d, var_list_d))
train_d = optimizer_d.apply_gradients(grads_and_vars_d,
global_step=tf.Variable(0, trainable=False, name="global_step_d"))
increment_t_d = tf.Variable(0, trainable=False, name="iteration_d").assign_add(1)

# loss and trainer for recognition network     
reg_terms_all = tf.losses.get_regularization_losses()
reg_terms = [r for r in reg_terms_all if r not in reg_terms_d]
scale = 1.0
scaled_ratio = tf.reduce_sum(scale * r_qsample)
pbeta_log_prob = 0.0
qbeta_log_prob = 0.0
loss = -(pbeta_log_prob - qbeta_log_prob + scaled_ratio -
         tf.reduce_sum(reg_terms))
var_list = [v for v in tf.trainable_variables() if v not in var_list_d]
print('var_list \n', var_list)

optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9) # optimizer for r(x,z)

grads = tf.gradients(loss, var_list)
grads_and_vars = list(zip(grads, var_list))
train = optimizer.apply_gradients(grads_and_vars,
global_step=tf.Variable(0, trainable=False, name="global_step"))
increment_t = tf.Variable(0, trainable=False, name="iteration").assign_add(1)


## start session
sess = ed.get_session()
tf.global_variables_initializer().run()
feed_dict = {x_ph : x_train}


# execute and visualize

- task is to match the samples of $q(z|x) q(x)$ and $p(z,x)$
- for $q(x)$ being the empirical distribution, this is achieved when $q(z|x) \approx p(z|x)$
- we're done once the samples from $q(z|x)$ (blue dots) match the samples from $p(z|x)$ (black dots)

In [ ]:
import time
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(12,14))    

plt.subplot(2,2,1)
plt_axes = [-1, 2, -1.5, 1.5]
plt_idx = np.random.choice(x_test.shape[0], M, replace=False)
plt.plot(x_test[plt_idx], qz.eval(session=sess, feed_dict={x_ph: x_test[plt_idx]}), 'bo')
plt.plot(x_test[plt_idx], z_test[plt_idx], 'ko')
plt.title('q(z | x) before training')
plt.xlabel('x')
plt.ylabel('z')
plt.legend(['est.', 'true'], loc=4)
plt.axis([-1, 2, -1.5, 1.5])



## actual fitting 

tstart = time.time()
lss_d = np.zeros(10000)
lss = np.zeros(10000)
for i in range(lss_d.size):
    
    idx = np.random.choice(N, M, replace=False)
    feed_dict = {x_ph : x_train[idx]}

    _, t, loss_d_ = sess.run(
              [train_d, increment_t_d, loss_d], feed_dict=feed_dict)
    lss_d[i] = loss_d_
    
    _, t, loss_ = sess.run(
              [train, increment_t, loss], feed_dict=feed_dict)    
    lss[i] = loss_
    
dur = time.time() - tstart

print('fitting took ' + str(dur) + 's')
 
    
    
plt.subplot(4,2,2)
plt.semilogx(lss_d)
plt.xlabel('iterations')
plt.ylabel('discriminator loss')
plt.subplot(4,2,4)
plt.semilogx(lss)
plt.xlabel('iterations')
plt.ylabel('recognition loss')

plt.subplot(2,2,3)
plt_axes = [-1, 2, -1.5, 1.5]
idx = np.random.choice(x_test.shape[0], M, replace=False)
plt.plot(x_test[plt_idx], qz.eval(session=sess, feed_dict={x_ph: x_test[plt_idx]}), 'bo')
plt.plot(x_test[plt_idx], z_test[plt_idx], 'ko')
plt.title('q(z | x) after training')
plt.xlabel('x')
plt.ylabel('z')
plt.legend(['est.', 'true'], loc=4)
plt.axis([-1, 2, -1.5, 1.5])

plt.subplot(2,2,4)
gx, gz = np.meshgrid(np.linspace(plt_axes[0], plt_axes[1], 25), np.linspace(plt_axes[2], plt_axes[3], 40))
feed_dict_r = {x_ph_t: gx.reshape(-1,1), z_ph_t: gz.reshape(-1,1)}
plt.imshow( np.flipud(r_test.eval(session=sess, feed_dict=feed_dict_r).reshape(40,25)), aspect='auto' ) 
plt.colorbar()
plt.title('learned log-density ratio r(x,z)')
plt.xlabel('x')
plt.ylabel('z')
plt.xticks([])
plt.yticks([])

plt.savefig('lfvi_prototype_logGaussianToyExample.pdf')

plt.show()
